In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# import os; os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [3]:
import numpy as np
import tensorflow as tf

import wandb
from wandb.keras import WandbCallback
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

from perceptnet.networks import *
from perceptnet.pearson_loss import PearsonCorrelation

from iqadatasets.datasets.tid2008 import TID2008
from iqadatasets.datasets.tid2013 import TID2013
from flayers.callbacks import *

# Load the data

In [4]:
cuac = TID2008("/lustre/ific.uv.es/ml/uv075/Databases/IQA/TID/TID2008", exclude_imgs=[25])
cuac_val = TID2013("/lustre/ific.uv.es/ml/uv075/Databases/IQA/TID/TID2013", exclude_imgs=[25])

In [5]:
dst_train = cuac.dataset
dst_val = cuac_val.dataset

2023-02-20 10:13:40.271297: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-02-20 10:13:40.271383: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: mlui02.ific.uv.es
2023-02-20 10:13:40.271402: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: mlui02.ific.uv.es
2023-02-20 10:13:40.271570: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 520.61.5
2023-02-20 10:13:40.271628: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 520.61.5
2023-02-20 10:13:40.271641: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 520.61.5
2023-02-20 10:13:40.273139: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CP

# Wandb config

In [6]:
id = "lmtpkz32" # GDNGaussian
# id = "0jz5nx0m" # GaborLast
MORE_EPOCHS = 0

In [16]:
wandb.init(project='PerceptNet2',
            id=id,
            mode="online",
            resume="allow",
            )
config = wandb.config
config.epochs += MORE_EPOCHS

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jorgvt. Use `wandb login --relogin` to force relogin


# Define the model

In [23]:
# model = PerceptNetExpGDNGaussian(kernel_initializer=config.kernel_initializer, gdn_kernel_size=config.gdn_kernel_size)
model = PerceptNetExpGaborLast(kernel_initializer=config.kernel_initializer, gdn_kernel_size=config.gdn_kernel_size)
model.build((None,384,512,3))
model.compile(optimizer=tf.optimizers.Adam(learning_rate=config.learning_rate),
              loss=PearsonCorrelation())

In [19]:
best_model = wandb.run.use_artifact(f"run_{wandb.run.id}_model:latest", type="model")
best_model_dir = best_model.download()

wandb:   3 of 3 files downloaded.  


In [24]:
model.load_weights(f"{best_model_dir}/model-best")

<class 'ValueError'>: Received incompatible tensor with shape (1, 1, 1, 3) when attempting to restore variable with shape (11, 11, 1, 3) and name feature_extractor/layer_with_weights-0/conv/kernel/.ATTRIBUTES/VARIABLE_VALUE.

In [1]:
summary = wandb.run.summary._as_dict()
summary["epoch/epoch"], summary["_step"]

(219, 219)

In [11]:
history = model.fit(dst_train.shuffle(buffer_size=100,
                                      reshuffle_each_iteration=True,
                                      seed=42) \
                             .batch(config.batch_size), 
                    epochs=config.epochs, 
                    initial_epoch=summary["epoch/epoch"]+1,
                    validation_data=dst_val.batch(config.batch_size),
                    callbacks=[WandbMetricsLogger(log_freq="epoch", initial_global_step=summary["_step"]+1),
                               WandbModelCheckpoint(filepath="model-best",
                                                    monitor="val_loss",
                                                    save_best_only=True,
                                                    save_weights_only=True,
                                                    mode="min")
                               ],
                    verbose=config.verbose)

<class 'ValueError'>: in user code:

    File "/lhome/ext/uv075/uv0752/miniconda3/envs/cuda/lib/python3.8/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/lhome/ext/uv075/uv0752/miniconda3/envs/cuda/lib/python3.8/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/lhome/ext/uv075/uv0752/miniconda3/envs/cuda/lib/python3.8/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/lhome/ext/uv075/uv0752/perceptnet/perceptnet/networks.py", line 44, in train_step
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
    File "/lhome/ext/uv075/uv0752/miniconda3/envs/cuda/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/optimizer_v2.py", line 678, in apply_gradients
        return tf.__internal__.distribute.interim.

wandb: WARNING When using `save_best_only`, ensure that the `filepath` argument contains formatting placeholders like `{epoch:02d}` or `{batch:02d}`. This ensures correct interpretation of the logged artifacts.


In [ ]:
wandb.finish()

<class 'TypeError'>: get_range() missing 1 required positional argument: 'session'